# import necessary libraries

In [1]:
# Loading Libraries in The Dataset
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix
from sklearn.metrics import classification_report,roc_curve, roc_auc_score,classification_report


from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
import re

## Loading NEO past  Data 

In [2]:
df_past=pd.read_csv('NEO Earth Close Approaches_past.csv')
df_past.head()

,Object,Close-Approach (CA) Date,CA DistanceNominal (au),CA DistanceMinimum (au),V relative(km/s),V infinity(km/s),H(mag),Diameter,Rarity
0,509352 (2007 AG),1900-Jan-04 22:25 ± 00:02,0.00963,0.00962,8.69,8.65,20.2,250 m - 550 m,2
1,(2014 SC324),1900-Jan-11 01:07 ± 00:18,0.03997,0.03991,10.65,10.65,24.4,35 m - 79 m,0
2,(2012 UK171),1900-Jan-12 23:07 ± 00:13,0.04982,0.04950,7.16,7.15,24.4,35 m - 77 m,0
3,(2024 BW1),1900-Jan-25 19:29 ± 4_11:16,0.03798,0.01013,6.34,6.33,25.1,25 m - 56 m,0
4,4660 Nereus (1982 DB),1900-Jan-29 18:09 ± 00:35,0.02080,0.02077,5.54,5.52,18.8,0.33±0.05 km,2


# Loading NEO Future  Data 

In [3]:
# Load the dataset
df_future=pd.read_csv('NEO_future.csv')
df_future.head()

,Object,Close-Approach (CA) Date,CA DistanceNominal (au),CA DistanceMinimum (au),V relative(km/s),V infinity(km/s),H(mag),Diameter,Rarity
0,(2024 KA1),2024-Jun-07 11:06 ± < 00:01,0.01255,0.01254,8.32,8.30,25.3,23 m - 51 m,0.0
1,(2024 LA),2024-Jun-07 22:20 ± < 00:01,0.01204,0.01197,18.28,18.27,25.8,18 m - 41 m,0.0
2,(2024 LC),2024-Jun-08 12:18 ± < 00:01,0.03137,0.03122,8.71,8.70,25.6,20 m - 45 m,0.0
3,(2024 CR9),2024-Jun-11 00:03 ± < 00:01,0.04924,0.04924,7.38,7.37,19.5,340 m - 760 m,1.0
4,(2024 LD),2024-Jun-11 11:06 ± 00:02,0.03093,0.03067,10.68,10.67,26.1,16 m - 36 m,0.0


# concatinating past and future data 

In [4]:
df=pd.concat([df_past,df_future],axis=0)
df=df.reset_index(drop=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29918 entries, 0 to 29917
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Object                    29918 non-null  object 
 1   Close-Approach (CA) Date  29918 non-null  object 
 2   CA DistanceNominal (au)   29918 non-null  float64
 3   CA DistanceMinimum (au)   29918 non-null  float64
 4   V relative(km/s)          29918 non-null  float64
 5   V infinity(km/s)          29910 non-null  float64
 6   H(mag)                    29916 non-null  float64
 7   Diameter                  29916 non-null  object 
 8   Rarity                    29916 non-null  float64
dtypes: float64(6), object(3)
memory usage: 2.1+ MB
